# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

In [107]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud_2020 #pip install git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas as pd #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [2]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [3]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [4]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xh8jfApAMFIAAJw8PzAAAADJ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":931112759,"wgRevisionId":931112759,"wgArticleId":473317,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Article

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

In [5]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dm


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [6]:
#content_analysis_save = 'wikipedia_content_analysis.html'
with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

Now lets open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag.

In [7]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Content analysis is a research method for studying documents and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.[1] One of the key advantages of using content analysis to analyse social phenomena is its non-invasive nature, in contrast to simulating social experiences or collecting survey answers.

Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful pieces of content.[2][3] By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts.

Computers are increasingly used in conten

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [8]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   Content analysis is a research method for stud...
1   Practices and philosophies of content analysis...
2   Computers are increasingly used in content ana...
3   Content analysis is best understood as a broad...
4   The simplest and most objective form of conten...
5   A further step in analysis is the distinction ...
6   More generally, content analysis is research u...
7   By having contents of communication available ...
8   Robert Weber notes: "To make valid inferences ...
9   There are five types of texts in content analy...
10  Over the years, content analysis has been appl...
11  In recent times, particularly with the advent ...
12  Quantitative content analysis has enjoyed a re...
13  Content analysis can also be described as stud...
14  Holsti groups fifteen uses of content analysis...
15  He also places these uses into the context of ...
16  The following table shows fifteen uses of cont...
17  The process of the initi

Now we have a `DataFrame` containing all relevant text from the page ready to be
processed

If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

In [9]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<_sre.SRE_Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [13]:
print(regexResults.group(0))

2134567890


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [15]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

In [65]:
wikipediabaseurl_clinderella = 'https://en.wikipedia.org/wiki/Cinderella'
wikiRequest = requests.get(wikipediabaseurl_clinderella)
wikiSoup = bs4.BeautifulSoup(wikiRequest.text, 'html.parser')
contentPTags = wikiSoup.body.findAll('p')
for pTag in contentPTags[7:]:
    print(pTag.text)

A version of the story, Ye Xian, appeared in Miscellaneous Morsels from Youyang written by Duan Chengshi around 860.[12] In this version, Ye Xian is the daughter of the local tribal leader who died when she was young. Because her mother died before her father, she is now under the care of her father's second wife, who abused her. She befriends a fish, which is the reincarnation of her deceased mother.[12] Her stepmother and half-sister kill the fish, but Ye Xian found the bones, which are magical, and they help her dress appropriately for a local Festival, including a very light golden shoe.[12] Her stepfamily recognizes her at the festival, causing her to flee and accidentally lose the shoe.  Afterwards, the king of another sea island obtained the shoe and is curious about the shoe as no one has feet that can fit the shoe. The King searched everywhere and finally came upon Ye's house. She tries on the shoe, and the king then realises she is the one and takes her back to his kingdom. H

In [90]:
contentPTags[31]

<p>Another interpretation of verre/vair (glass/fur) suggested a sexual element—the Prince was 'trying on' the 'fur slipper' (vagina) of the maidens in the kingdom, as a '<a href="/wiki/Droit_du_seigneur" title="Droit du seigneur">Droit du seigneur</a>' right of sexual possession of his subjects. The disguised Cinderella's 'fur slipper' was of unique appeal to the Prince who sought her thereafter through sexual congress (a variety of sources including Joan Gould).
</p>

In [125]:
re.sub(r'\\', '','i\'s')

"i's"

In [126]:
contentParagraphs = []
for pTag in contentPTags:
    clean = re.sub(r'\[\d+\]', '', pTag.text)# remove the citation and []
    clean = re.sub(r'\[.*\]','',clean)# remove the ['string']
    clean = re.sub('Plot:','',clean)
    clean = re.sub('Notes','',clean)
    clean = re.sub(r'/',' or ',clean)
    clean = re.sub(r'\(.*\)','',clean)
    clean = re.sub(r'\n','',clean)
    clean = re.sub(r'\\','', clean)
    #clean = clean.strip(' ').strip('\n')
    if clean == '':
        continue
    contentParagraphs.append(clean)

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   "Cinderella", or "The Little Glass Slipper", i...
1   The Chinese story of Ye Xian, first attested i...
2   Although the story's title and main character'...
3   The oldest known oral version of the Cinderell...
4   The same story is also later reported by the R...
5   Herodotus, some five centuries before Strabo, ...
6   The twelfth-century AD lai of Le Fresne . The ...
7   A version of the story, Ye Xian, appeared in M...
8   The existing version of the Japanese tale date...
9   Several different variants of the story appear...
10  The Story of Tam and Cam, from Vietnam, is sim...
11  The Moon Brow   is the Iranian version of the ...
12  The first written European version of the stor...
13  The name "Cenerentola" comes from the Italian ...
14  Giambattista Basile, an Italian soldier and go...
15  One of the most popular versions of Cinderella...
16  The first moral of the story is that beauty is...
17  However, the second mora

In [91]:
len(contentParagraphs)

43


# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [92]:
#wikipedia_base_url = 'https://en.wikipedia.org'
otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Folklore', 0, 'folk tale'), ('https://en.wikipedia.org/wiki/Myth', 0, 'myth'), ('https://en.wikipedia.org/wiki/Rhodopis', 0, 'Rhodopis'), ('https://en.wikipedia.org/wiki/Strabo', 0, 'Strabo'), ('https://en.wikipedia.org/wiki/Italy', 1, 'Italy'), ('https://en.wikipedia.org/wiki/Giambattista_Basile', 1, 'Giambattista Basile'), ('https://en.wikipedia.org/wiki/Pentamerone', 1, 'Pentamerone'), ('https://en.wikipedia.org/wiki/Charles_Perrault', 1, 'Charles Perrault'), ('https://en.wikipedia.org/wiki/Histoires_ou_contes_du_temps_pass%C3%A9', 1, 'Histoires ou contes du temps passé'), ('https://en.wikipedia.org/wiki/Brothers_Grimm', 1, 'Brothers Grimm')]


We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [17]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,Content analysis is a research method for stud...,https://en.wikipedia.org/wiki/Content_analysis,0
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4
5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,5
6,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,6
7,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,7
8,"Robert Weber notes: ""To make valid inferences ...",https://en.wikipedia.org/wiki/Content_analysis,8
9,There are five types of texts in content analy...,https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [18]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [19]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,Content analysis is a research method for stud...,https://en.wikipedia.org/wiki/Content_analysis,None,None
1,1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,None,None
2,2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,None,None
3,3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,None,None
4,4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,None,None
5,5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
6,6,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,None,None
7,7,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
8,8,"Robert Weber notes: ""To make valid inferences ...",https://en.wikipedia.org/wiki/Content_analysis,None,None
9,9,There are five types of texts in content analy...,https://en.wikipedia.org/wiki/Content_analysis,None,None



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



In [97]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipediabaseurl_clinderella = 'https://en.wikipedia.org/wiki/Cinderella'
wikiRequest = requests.get(wikipediabaseurl_clinderella)
wikiSoup = bs4.BeautifulSoup(wikiRequest.text, 'html.parser')
contentPTags = wikiSoup.body.findAll('p')

In [94]:
first_otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        first_otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
    

In [128]:
def clean_context(pTag):
    clean = re.sub(r'\[\d+\]', '', pTag.text)# remove the citation and []
    clean = re.sub(r'\[.*\]','',clean)# remove the ['string']
    clean = re.sub('Plot:','',clean)
    clean = re.sub('Notes','',clean)
    clean = re.sub(r'/',' or ',clean)
    clean = re.sub(r'\(.*\)','',clean)
    clean = re.sub(r'\n','',clean)
    clean = re.sub(r'\\','', clean)
        #clean = clean.strip(' ').strip('\n')
    return clean

In [136]:
def getTextFromWikiPage(targetURL):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-text' : [], 'paragraph_num':[]}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        if clean_context(pTag) == '':
            continue
        parsDict['paragraph-text'].append(clean_context(pTag))
        
        parsDict['source'].append(targetURL)
        parsDict['paragraph_num'].append(parNum)
    return pandas.DataFrame(parsDict)

In [100]:
def get_page_hyperlink(base_url,source_url):
    wikiRequest = requests.get(source_url)
    wikiSoup = bs4.BeautifulSoup(wikiRequest.text, 'html.parser')
    contentPTags = wikiSoup.body.findAll('p')
    otherPAgeURLS = []
    content_list = []
    for paragraphNum, pTag in enumerate(contentPTags):
        #we only want hrefs that link to wiki pages
        tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
        for aTag in tagLinks:
            #We need to extract the url from the <a> tag
            relurl = aTag.get('href')
            linkText = aTag.text
            #wikipedia_base_url is the base we can use the urllib joining function to merge them
            #Giving a nice structured tupe like this means we can use tuple expansion later
            otherPAgeURLS.append(
                urllib.parse.urljoin(base_url, relurl)
            )
        content_list.append(clean_context(pTag))
    return otherPAgeURLS, content_list 

In [101]:
def recursive_spider(source_url, max_depth,url_list):
    if max_depth == 0:
        url_list.extend(get_page_hyperlink(wikipedia_base_url,source_url))
        return list(set(url_list))
    else:
        next_find = get_page_hyperlink(wikipedia_base_url,source_url)
        url_list.extend(next_find)
        for url in next_find:
            url_list = recursive_spider(url, max_depth-1,url_list)
    return list(set(url_list))    

In [105]:
url_list = []
url_list = recursive_spider(wikipediabaseurl_clinderella, 1, url_list)

In [104]:
url_list

['https://en.wikipedia.org/wiki/Folklore',
 'https://en.wikipedia.org/wiki/Myth',
 'https://en.wikipedia.org/wiki/Rhodopis',
 'https://en.wikipedia.org/wiki/Strabo',
 'https://en.wikipedia.org/wiki/Italy',
 'https://en.wikipedia.org/wiki/Giambattista_Basile',
 'https://en.wikipedia.org/wiki/Pentamerone',
 'https://en.wikipedia.org/wiki/Charles_Perrault',
 'https://en.wikipedia.org/wiki/Histoires_ou_contes_du_temps_pass%C3%A9',
 'https://en.wikipedia.org/wiki/Brothers_Grimm',
 'https://en.wikipedia.org/wiki/Grimms%27_Fairy_Tales',
 'https://en.wikipedia.org/wiki/Trope_(literature)',
 'https://en.wikipedia.org/wiki/Rhodopis',
 'https://en.wikipedia.org/wiki/Greeks_in_Egypt',
 'https://en.wikipedia.org/wiki/Hetaira',
 'https://en.wikipedia.org/wiki/Colonies_in_antiquity',
 'https://en.wikipedia.org/wiki/Naucratis',
 'https://en.wikipedia.org/wiki/Ancient_Egypt',
 'https://en.wikipedia.org/wiki/Ancient_Greece',
 'https://en.wikipedia.org/wiki/Strabo',
 'https://en.wikipedia.org/wiki/Memphi

In [210]:
url_list = list(set(url_list))
contentParagraphsDF = []
for url in url_list[:10]:
    contentParagraphsDF.append(getTextFromWikiPage(url))
whole = pd.concat(contentParagraphsDF)
whole

,paragraph-text,paragraph_num,source
0,The Raeti or ˈriːtaɪ or were a confederatio...,0,https://en.wikipedia.org/wiki/Rhaetian_people
1,The etymology of the name Raeti is uncertain. ...,1,https://en.wikipedia.org/wiki/Rhaetian_people
2,Ancient sources characterise the Raeti as Etru...,2,https://en.wikipedia.org/wiki/Rhaetian_people
3,"The Raeti were divided into numerous tribes, b...",3,https://en.wikipedia.org/wiki/Rhaetian_people
4,"The Raeti tribes, together with those of their...",4,https://en.wikipedia.org/wiki/Rhaetian_people
5,The origin of the name Raeti is uncertain. It ...,5,https://en.wikipedia.org/wiki/Rhaetian_people
6,It was suggested in the 19th century that Raet...,6,https://en.wikipedia.org/wiki/Rhaetian_people
7,"The Roman geographer Pliny the Elder, writing ...",7,https://en.wikipedia.org/wiki/Rhaetian_people
8,It has also been suggested that the name Raeti...,8,https://en.wikipedia.org/wiki/Rhaetian_people
9,The earliest mention of the Raeti in surviving...,9,https://en.wikipedia.org/wiki/Rhaetian_people


## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [155]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":3465,"posts-type":false,"posts":[{"id":"190295315782","url":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/190295315782","url-with-slug":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/190295315782\/no-thanks","type":"photo","date-gmt":"2020-01-16 21:01:32 GMT","date":"Thu, 16 Jan 2020 22:01:32","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1579208492,"format":"html","reblog-key":"sRdYDpGd","slug":"no-thanks","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"190295315782\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_190295315782\"><iframe id=\"like_iframe_190295315782\" src=\"https:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=66c22ab5319d742bca5762b8d18f9d06#name=lolcats-lol-cat&amp;post_id=190295315782&amp;color=black&a

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [156]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [158]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [159]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,date,id,photo-type,photo-url,tags
0,"Thu, 16 Jan 2020 22:01:32",190295315782,jpg,https://66.media.tumblr.com/60a2b6d7356f547032...,"[cat, cats, lol, lolcat, lolcats]"
1,"Thu, 16 Jan 2020 10:01:16",190287026776,png,https://66.media.tumblr.com/6f26a4d5b19774f06c...,"[cat, cats, lol, lolcat, lolcats]"
2,"Mon, 13 Jan 2020 12:03:57",190233110922,jpg,https://66.media.tumblr.com/42458742dedc510f3d...,"[cat, cats, lol, lolcat, lolcats]"
3,"Mon, 13 Jan 2020 10:00:35",190232075547,jpg,https://66.media.tumblr.com/65b1e6efd6ec145872...,"[cat, cats, lol, lolcat, lolcats]"
4,"Mon, 13 Jan 2020 08:01:02",190230902981,jpg,https://66.media.tumblr.com/616b4faab5a289a369...,"[cat, cats, lol, lolcat, lolcats]"
5,"Mon, 13 Jan 2020 06:00:57",190229342226,jpg,https://66.media.tumblr.com/28121ad01b05305b89...,"[cat, cats, lol, lolcat, lolcats]"
6,"Sun, 12 Jan 2020 06:01:05",190209640041,jpg,https://66.media.tumblr.com/9b735740f517f3673b...,"[cat, cats, lol, lolcat, lolcats]"
7,"Sat, 11 Jan 2020 10:01:29",190193496849,jpg,https://66.media.tumblr.com/1ffa3c880a55f80f62...,"[cat, cats, lol, lolcat, lolcats]"
8,"Sat, 11 Jan 2020 08:00:47",190192270965,jpg,https://66.media.tumblr.com/a5ecfce3b06defdbac...,"[cat, cats, lol, lolcat, lolcats]"
9,"Sat, 11 Jan 2020 06:01:02",190190785530,jpg,https://66.media.tumblr.com/522386ce0644901d60...,"[cat, cats, lol, lolcat, lolcats]"


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [160]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [161]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. LEts tart by looking at the Shakespeare files in the `data` directory 

In [162]:
with open('../data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

FileNotFoundError: [Errno 2] No such file or directory: '../data/Shakespeare/midsummer_nights_dream.txt'

Then to load all the files in `../data/Shakespeare` we can use a for loop with `scandir`:

In [ ]:
targetDir = '../data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path) as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [ ]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [138]:
information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.5
%����
97 0 obj
<<
/Length 2988      
/Filter /FlateDecode
>>
stream
xڝYY��~�_��T,U�����f<{����z\[)o(
���!�������n��4�J�/b�� }7�9�s������o���I�,�|�a�d�J��M��y�9?�~<i]כHE���	�}����e��k��esp�ߋ�շyS�Jf��|�?>���׺�+��y7����==w��8�����M<�x�
߉�M�Q�k�k�tW�C�6덂
������拍���Y��o乤�8s���Fy-�����g��vGvM ����3���}���[+o�y��שnٰ|7N6UŽ�f����l����D�D��|[i�n��MY��f�3��W�n`�>���X!~lڧ��>��K�7�
��P�z������s;
�zx�����t��|ӎ���n���V{�Av@�Wt:�!bc!*���>����΋�@R�Q`��_�A���Aл�yG+J���z[��\[G�1-r��U�m�A�|��'�2


It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [139]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [140]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [141]:
print(readPDF(infoExtractionBytes)[:550])

Speech and Language Processing. Daniel Jurafsky & James H. Martin.
rights reserved.

Draft of November 7, 2016.

Copyright c(cid:13) 2016.

All

CHAPTER

21 Information Extraction

I am the very model of a modern Major-General,
I’ve information vegetable, animal, and mineral,
I know the kings of England, and I quote the ﬁghts historical
From Marathon to Waterloo, in order categorical...
Gilbert and Sullivan, Pirates of Penzance

Imagine that you are an analyst with an investment ﬁrm that tracks airline stocks.
You’re given the task of determini


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [ ]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud_2020` so we can use it later without having to retype it.

In [ ]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [ ]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [ ]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


In [168]:
information_extraction_pdf = 'https://www.bartleby.com/ebook/adobe/3134.pdf'
infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%����
<< /Type /MCR /Pg 57 0 R 


In [143]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

In [169]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

In [170]:
print(readPDF(infoExtractionBytes)[:550])

Five Short Stories
Alphonse Daudet

The Harvard Classics Shelf of Fiction, Vol. XIII, Part 4.

Selected by Charles William Eliot

Copyright © 2001 Bartleby.com, Inc.

Bibliographic Record

Contents

Biographical Note
Criticisms and Interpretations
I. By Henry James
II. By George Pellissier

1. The Siege of Berlin
2. The Last Class—The Story of a Little Alsatian
3. The Child Spy
4. The Game of Billiards
5. The Bad Zouave

Biographical Note

ALPHONSE DAUDET was born at Nîmes in the south of France on May 13, 1840. His father was an
unsuccessful s


In [171]:
string = readPDF(infoExtractionBytes)

In [182]:
string

'Five Short Stories\nAlphonse Daudet\n\nThe Harvard Classics Shelf of Fiction, Vol. XIII, Part 4.\n\nSelected by Charles William Eliot\n\nCopyright © 2001 Bartleby.com, Inc.\n\nBibliographic Record\n\nContents\n\nBiographical Note\nCriticisms and Interpretations\nI. By Henry James\nII. By George Pellissier\n\n1. The Siege of Berlin\n2. The Last Class—The Story of a Little Alsatian\n3. The Child Spy\n4. The Game of Billiards\n5. The Bad Zouave\n\nBiographical Note\n\nALPHONSE DAUDET was born at Nîmes in the south of France on May 13, 1840. His father was an\nunsuccessful silk manufacturer, and his boyhood was far from happy. After a period of schooling at\nLyons, he became at sixteen usher in a school, but before the end of the following year he abandoned a\nprofession in which he found only misery. Going up to Paris he joined his elder brother, Ernest, who was\nthen trying to get a foothold in journalism. At eighteen he published a volume of poems, “Les\nAmoureuses,” wrote for the “Fig

In [183]:
string.find('\n\nFootnotes\n\n')

61798

In [184]:
point = ['1. The Siege of Berlin','2. The Last Class—The Story of a Little Alsatian',\
        '3. The Child Spy','4. The Game of Billiards','5. The Bad Zouave','Footnotes']

In [193]:
re.sub(r'\d\. ' ,'','1. The Siege of Berlin')

'The Siege of Berlin'

In [197]:
string.find(post_po)

315

In [209]:
whole_story = pd.DataFrame()
story = []
for i in range(len(point)-1):
    pre_po = '\n\n'+point[i]+'\n\n'
    post_po = '\n\n'+point[i+1]+'\n\n'
    content = string[string.find(pre_po):string.find(post_po)]
    content = content.split('\n\n')
    paragraph = list(filter(lambda x: x!='',content))
    df_pdf = pd.DataFrame({'paragraph-text' : paragraph, 'paragraph_Num':range(len(paragraph)),\
                           'story':len(paragraph)*[re.sub(r'\d\. ','',point[i])]})
    story.append(df_pdf)
whole_story = pd.concat(story)
whole_story

,paragraph-text,paragraph_Num,story
0,1. The Siege of Berlin,0,The Siege of Berlin
1,WE were going up Avenue des Champs-Elysées wit...,1,The Siege of Berlin
2,“Do you see those four closed windows up the...,2,The Siege of Berlin
3,“I found the former cuirassier stretched out...,3,The Siege of Berlin
4,“The child’s grief touched me. Daughter and ...,4,The Siege of Berlin
5,less terrible. I comforted her as best I coul...,5,The Siege of Berlin
6,“‘Vic-to-ry!’,6,The Siege of Berlin
7,“And as I gave him details of the grand expl...,7,The Siege of Berlin
8,"“When I left the room, the girl was waiting ...",8,The Siege of Berlin
9,"“‘But he is saved!’ I said, taking her hands.",9,The Siege of Berlin


### Other sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

